## Create a Databricks notebook to load the Silver layer: Customer delta table

In [0]:
spark.sql("USE globalretail_silver")
spark.sql("""
    CREATE TABLE IF NOT EXISTS silver_customers(
      customer_id STRING,
      name STRING,
      email STRING,
      country STRING,
      customer_type STRING,
      registration_date DATE,
      age INTEGER,
      gender STRING,
      total_purchases INTEGER,  
      customer_segment STRING,
      days_since_registration INTEGER,
      last_updated TIMESTAMP
    )
""")

- Whenever new data arrives in the silver layer table, we need to distinguish between fresh records from the bronze layer and existing records. 
- To do this, we simply identify the latest updated timestamp in the silver layer table.

In [0]:
# Get the most recent last_updated timestamp from the silver_customers table
last_processed_df = spark.sql("SELECT MAX(last_updated) AS last_processed FROM silver_customers")
last_processed_timestamp = last_processed_df.collect()[0]['last_processed']

# If no records exist, set a default old timestamp
if last_processed_timestamp is None:
  last_processed_timestamp = "1900-01-01T00:00:00.000+0000"

In [0]:
# Create a temporary view of incremental data
spark.sql(f"""
    CREATE OR REPLACE TEMPORARY VIEW bronze_incremental_customers AS
    SELECT * 
    FROM globalretail_bronze.bronze_customers c
    WHERE c.ingestion_timestamp >= '{last_processed_timestamp}'
""")

In [0]:
spark.sql("select * from bronze_incremental").show()

- **Data Transformation:**
  - Ensure email addresses are not null
  - Validate that age is between 18 and 100
  - Assign customer_segment: 'High Value' if total_purchases > 10,000, 'Medium Value' if > 5,000, otherwise 'Low Value'
  - Calculate days since the user registered in the system
  - Exclude records where total_purchases is negative

In [0]:
# Transform data

spark.sql("""
    CREATE OR REPLACE TEMPORARY VIEW silver_incremental_customers AS
    SELECT 
      customer_id,
      name,
      email,
      country,  
      customer_type,
      registration_date,
      age,
      gender,
      total_purchases,
      CASE 
        WHEN total_purchases > 10000 THEN 'High Value'
        WHEN total_purchases > 5000 THEN 'Medium Value'
        ELSE 'Low Value'
      END AS customer_segment,
      datediff(current_date(), registration_date) AS days_since_registration,
      current_timestamp() AS last_updated
    FROM bronze_incremental
    WHERE
        age BETWEEN 18 AND 100 AND
        total_purchases >= 0 AND
        email IS NOT NULL
""")

In [0]:
display(spark.sql("select * from silver_incremental_customers"))

In [0]:
spark.sql("""
    MERGE INTO silver_customers t
    USING silver_incremental_customers s
    ON t.customer_id = s.customer_id
    WHEN MATCHED THEN
      UPDATE SET *
    WHEN NOT MATCHED THEN
      INSERT *
""")

In [0]:
spark.sql("select count(*) from silver_customers").show()